# Computer Vision Manipulations

In [ ]:
import cv2
import numpy as np
import itertools as it
import os
import sys
import multiprocessing as mp
import pandas as pd

## Functions

In [2]:
def createHash(img, bot, top):
    hashmap = {}
    for i,j in it.product(range(img.shape[0]), range(img.shape[1])):
        if img[i,j] not in hashmap:
            if img[i,j] <= bot:
                hashmap[img[i,j]] = 0
            elif img[i,j] > top:
                hashmap[img[i,j]] = 255
            else:
                hashmap[img[i,j]] = 255 * ((img[i,j] - bot) / (top - bot))
                
    return hashmap

def grayscale(img):
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [3]:
def linearStretch(img):
    bot = min(img.ravel())
    top = max(img.ravel())
        
    hashmap = createHash(img, bot, top)
    img2 = img
    for i, j in it.product(range(img2.shape[0]), range(img2.shape[1])):
        img2[i,j] = hashmap[img2[i,j]]
        
    return img2

def createROI(img, h1, h2, w1, w2):
    
    return img[h1:h2, w1:w2]

def powerStretch(img):
    img2 = img
    #img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)  
    img2 = img2/255.0      
    im_power_law_transformation = cv2.pow(img2,2.5)
    im_power_law_transformation *= 255 
    im_power_law_transformation = im_power_law_transformation.astype('uint8')
    #im_power_law_transformation = cv2.cvtColor(im_power_law_transformation,cv2.COLOR_BGR2RGB)
        
    return im_power_law_transformation

def integratedColorModel(img):
    hsvimg = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)    
    hue = hsvimg[:,:,0]
    saturation = hsvimg[:,:,1]
    value = hsvimg[:,:,2]
    valuecs = linearStretch(value)
    saturationcs = linearStretch(saturation)
    hlscs = cv2.merge((hue,saturationcs,valuecs))
    imc = cv2.cvtColor(hlscs, cv2.COLOR_HSV2RGB)
   
    return imc

In [11]:
def getImagesPath(parent_path, child_path, filenames):
    paths = []
    for f in filenames:
        folder = f[:f.find('frame')-1]
        framename = 'wb_' + f[f.find('frame'):]
        if os.path.exists(parent_path + folder + child_path + framename):
            paths.append((folder,framename))
        
    return paths       

In [12]:
parent_path = '/home/diegues/2018-05-01-OMARE/logs/lauv-noptilus-3/20180501/'
child_path = '/mra/FilteredPhotos/WhiteBalance/'

df_path = "/home/diegues/Desktop/ProcessedImages/sampled_data.csv"
data = pd.read_csv(df_path)

filenames = data['filename']
image_paths = getImagesPath(parent_path, child_path, filenames)
image_paths[:5]

[('113610_cam_survey_1', 'wb_frame2013.jpg'),
 ('113610_cam_survey_1', 'wb_frame2676.jpg'),
 ('113610_cam_survey_1', 'wb_frame962.jpg'),
 ('132143_forcadinho-np3', 'wb_frame1283.jpg'),
 ('113610_cam_survey_1', 'wb_frame699.jpg')]

In [13]:
cv_path = '/'.join(df_path.split('/')[:-1]) + '/CV/'
icm_path = '/icm/'
lcs_path = '/lcs/'
pcs_path = '/pcs/'
if not os.path.exists(cv_path):
    os.mkdir(cv_path)
for folder in [f[0] for f in image_paths]:
    if not os.path.exists(cv_path + folder):
        os.mkdir(cv_path + folder)
        os.mkdir(cv_path + folder + icm_path)
        os.mkdir(cv_path + folder + lcs_path)
        os.mkdir(cv_path + folder + pcs_path)


In [14]:
def createImages(t):
    img = cv2.imread(parent_path + t[0] + child_path + t[1])
    icm = integratedColorModel(img)
    lcs = linearStretch(grayscale(img))
    pcs = powerStretch(grayscale(img))
    cv2.imwrite(cv_path + t[0] + icm_path + t[0] + t[1][t[1].find('_'):], icm)
    cv2.imwrite(cv_path + t[0] + lcs_path + t[0] + t[1][t[1].find('_'):], lcs)
    cv2.imwrite(cv_path + t[0] + pcs_path + t[0] + t[1][t[1].find('_'):], pcs)

In [ ]:
pool = mp.Pool(processes=mp.cpu_count())
pool.map(createImages, image_paths)

In [60]:
t = image_paths[0]
cv_path + t[0] + icm_path + t[0] + t[1][t[1].find('_'):]

'/home/diegues/Desktop/ProcessedImages/CV/113610_cam_survey_1/icm/113610_cam_survey_1_frame2013.jpg'